# Eletricidade - Clusterização :

#### Marcio Carvalho

### Problema de negócio: 

##### Aneel - órgão regulador. Sabemos o consumo por estado, não conhecemos o perfil do cliente. Precisamos agrupar o perfil do cliente de acordo com o nível de consumo independe de onde eles moram, pois, queremos fazer uma atividade de marketing direcionada. Precisamos construir uma Máquina Preditiva (MP), a partir do consumo de energia de cliente. Agrupe os consumidores por similaridades, afim de compreender o comportamento dos clientes e sua relação com o consumo de energia (Simulação como se cada linha representasse vários clientes).

### Coleta:

https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

### Dataset:
         
##### Este arquivo contém 2.075.259 medições coletadas em uma casa localizada em Sceaux (7km de Paris, França) entre dezembro de 2006 e novembro de 2010 (47 meses).

### Features:

1.data: Data no formato dd/mm/aaaa

2.time: hora no formato hh:mm:ss

3.global_active_power: potência ativa média global doméstica por minuto (em quilowatts)

4.global_reactive_power: potência reativa média global doméstica por minuto (em quilowatts)

5.tensão: tensão média por minuto (em volts)

6.global_intensity: intensidade de corrente média global por minuto doméstica (em amperes)

7.sub_metering_1: sub medição de energia nº 1 (em watt-hora de energia ativa). Corresponde à cozinha, contendo maioritariamente uma máquina de lavar louça, um forno e um micro-ondas (as placas não são eléctricas mas sim a gás).

8.sub_metering_2: sub medição de energia nº 2 (em watt-hora de energia ativa). Corresponde à lavandaria, contendo máquina de lavar roupa, máquina de secar roupa, frigorífico e luz.

9.sub_metering_3: sub medição de energia nº 3 (em watt-hora de energia ativa). Corresponde a um esquentador eléctrico e a um ar condicionado.


     

   

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

C:\Users\carva\AppData\Local\Temp\ipykernel_2828\3228307996.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [23]:
# arquivos em diretórios diferentes, utilizado caminho relativo
df = pd.read_table('../data/household_power_consumption.txt', sep=";")
# renomear colunas com títulos mais intuitivos e traduzidos
df.columns = ['Data','Tempo','Potencia_ativa_total','Potencia_reativa_total','Tensao','Corrente_Total','Sub_area_cozinha','Sub_area_lavanderia','Sub_area_Aq_Ar']
# configurar o pandas para não dgerar saídas por notação científica e sim float com duas casas decimais
pd.set_option('display.float_format', '{:.2f}'.format)
df.head()

,Data,Tempo,Potencia_ativa_total,Potencia_reativa_total,Tensao,Corrente_Total,Sub_area_cozinha,Sub_area_lavanderia,Sub_area_Aq_Ar
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.00
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.00
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.00
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.00
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.00


In [42]:
df_cop = df.copy()

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Data                    object 
 1   Tempo                   object 
 2   Potencia_ativa_total    object 
 3   Potencia_reativa_total  object 
 4   Tensao                  object 
 5   Corrente_Total          object 
 6   Sub_area_cozinha        object 
 7   Sub_area_lavanderia     object 
 8   Sub_area_Aq_Ar          float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


In [40]:
df_cop.isnull().sum()

Data                          0
Tempo                         0
Potencia_ativa_total          0
Potencia_reativa_total        0
Tensao                        0
Corrente_Total                0
Sub_area_cozinha              0
Sub_area_lavanderia           0
Sub_area_Aq_Ar            25979
dtype: int64

In [43]:
# Verificar se cada célula contém '?', inicialmente foi gerado erro na tentativa de converter objeto em número. explicação do erro apresentou a string "?"
erro = df_cop.applymap(lambda x: '?' in str(x))
# Iterar sobre as colunas com '?' e substituir '?' por NaN e converter para float
for coluna in erro.columns:
    if erro[coluna].any():
        df_cop[coluna] = pd.to_numeric(df_cop[coluna], errors='coerce')
        print(f'A coluna {coluna} contém pelo menos um ponto de interrogação.')
df_cop.info()

A coluna Potencia_ativa_total contém pelo menos um ponto de interrogação.
A coluna Potencia_reativa_total contém pelo menos um ponto de interrogação.
A coluna Tensao contém pelo menos um ponto de interrogação.
A coluna Corrente_Total contém pelo menos um ponto de interrogação.
A coluna Sub_area_cozinha contém pelo menos um ponto de interrogação.
A coluna Sub_area_lavanderia contém pelo menos um ponto de interrogação.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Data                    object 
 1   Tempo                   object 
 2   Potencia_ativa_total    float64
 3   Potencia_reativa_total  float64
 4   Tensao                  float64
 5   Corrente_Total          float64
 6   Sub_area_cozinha        float64
 7   Sub_area_lavanderia     float64
 8   Sub_area_Aq_Ar          float64
dtypes: float64(7), object(2)
memory usage: 142.5+ MB


In [44]:
# Lista de colunas e seus respectivos formatos de data/tempo
colunas_formatos = {'Data': '%d/%m/%Y', 'Tempo': '%H:%M:%S'}
# Iterar sobre as colunas e formatos
for coluna, formato in colunas_formatos.items():
    df_cop[coluna] = pd.to_datetime(df_cop[coluna], format=formato)
df_cop.info()

In [46]:
df_cop.isnull().sum()

Data                          0
Tempo                         0
Potencia_ativa_total      25979
Potencia_reativa_total    25979
Tensao                    25979
Corrente_Total            25979
Sub_area_cozinha          25979
Sub_area_lavanderia       25979
Sub_area_Aq_Ar            25979
dtype: int64

In [47]:
df_cop.describe()

,Data,Tempo,Potencia_ativa_total,Potencia_reativa_total,Tensao,Corrente_Total,Sub_area_cozinha,Sub_area_lavanderia,Sub_area_Aq_Ar
count,2075259,2075259,2049280.00,2049280.00,2049280.00,2049280.00,2049280.00,2049280.00,2049280.00
mean,2008-12-05 19:13:27.255191040,1900-01-01 11:59:32.744808704,1.09,0.12,240.84,4.63,1.12,1.30,6.46
min,2006-12-16 00:00:00,1900-01-01 00:00:00,0.08,0.00,223.20,0.20,0.00,0.00,0.00
25%,2007-12-12 00:00:00,1900-01-01 06:00:00,0.31,0.05,238.99,1.40,0.00,0.00,0.00
50%,2008-12-06 00:00:00,1900-01-01 12:00:00,0.60,0.10,241.01,2.60,0.00,0.00,1.00
75%,2009-12-01 00:00:00,1900-01-01 18:00:00,1.53,0.19,242.89,6.40,0.00,1.00,17.00
max,2010-11-26 00:00:00,1900-01-01 23:59:00,11.12,1.39,254.15,48.40,88.00,80.00,31.00
std,NaN,NaN,1.06,0.11,3.24,4.44,6.15,5.82,8.44


In [48]:
# Substituir os valores ausentes nas colunas float pela mediana de cada coluna float
float_columns = df_cop.select_dtypes(include='float').columns
df_cop[float_columns] = df_cop[float_columns].fillna(df_cop[float_columns].median())
df_cop.isnull().sum()

Data                      0
Tempo                     0
Potencia_ativa_total      0
Potencia_reativa_total    0
Tensao                    0
Corrente_Total            0
Sub_area_cozinha          0
Sub_area_lavanderia       0
Sub_area_Aq_Ar            0
dtype: int64